# OpenPTV calibration using given markers and liboptv

In [13]:
# liboptv is different from openptv_python cause it's a C library with Cython bindings. requires different installation

In [2]:
# %pip install pyptv ipykernel plotly

In [3]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

import pathlib, os
notebook_path = pathlib.Path.cwd()
print(notebook_path)

/home/user/Documents/repos/proPTV_OpenPTV_MyPTV_comparison/OpenPTV_LineOfSight


In [16]:
class Parameter:
    cams = [0,1,2,3]
    Vmin = [0,0,0]
    Vmax = [300,300,300]
    N1, N2 = 361, 5

In [17]:
params = Parameter()

markers = [np.loadtxt('../proPTV_LineOfSight/markers_c'+str(cam)+'.txt') for cam in params.cams]
XYZ = markers[0][:,2:]
xy = markers[0][:,:2]
ID = np.argwhere((XYZ[:,0]>-1))[:,0]

FileNotFoundError: ../proPTV_LineOfSight/markers_c0.txt not found.

In [ ]:
xyz = pd.DataFrame(XYZ, columns=['x','y','z'])
xyz['id'] = ID
# px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()

In [ ]:
# First, let's calibrate roughly the cameras

In [ ]:

ref_pts = XYZ[[0,721,1409,1462],:]
xyz = pd.DataFrame(ref_pts, columns=['x','y','z'])
xyz['id'] = ID[[0,721,1409,1462]]
# px.scatter_3d(x=xyz['x'], y=xyz['y'], z=xyz['z'], color=xyz['id']).show()


In [ ]:
cam_id = 1 # or 2,3,4

In [8]:
# cal = Calibration().from_file(working_path / "calibration" / f"cam{cam_id}.tif.ori", None)
# cpar = ControlPar().from_file(working_path / "parameters" / "ptv.par")
# vpar = VolumePar().from_file(working_path / "parameters" / "criteria.par")

n_cams = 4

from optv.parameters import ControlParams, VolumeParams
cpar = ControlParams(n_cams)

if (notebook_path.drive == 'C:' or str(notebook_path).startswith('C:\\')):
    working_folder = pathlib.Path(r'C:\Users\Alex L\Dropbox\Open_Pro_My_PTV\Tracking\50000_30')
else:
    working_folder = pathlib.Path('/home/user/Dropbox/Open_Pro_My_PTV/Tracking/50000_30')
# working_folder = pathlib.Path(r'C:\Users\Alex L\Dropbox\Open_Pro_My_PTV\Tracking\50000_30')
# working_folder = pathlib.Path('/home/user/Dropbox/Open_Pro_My_PTV/Tracking/50000_30')

print(working_folder)

os.chdir(working_folder)

cpar.read_control_par("parameters/ptv.par")

vpar = VolumeParams()
vpar.read_volume_par("parameters/criteria.par")

from optv.calibration import Calibration

# Calibration parameters
cals = []
for i_cam in range(n_cams):
    cal = Calibration()
    tmp = cpar.get_cal_img_base_name(i_cam)
    cal.from_file(tmp + ".ori", tmp + ".addpar")
    print(cal.get_pos(), cal.get_angles())
    cals.append(cal)

/home/user/Dropbox/Open_Pro_My_PTV/Tracking/50000_30
[ 322.38251339    4.4040676  1031.76370716] [ 0.1884708   0.20904414 -0.03438548]
[-1.72196260e-01  2.96220478e+02  1.03049543e+03] [-0.2028321  -0.18304368 -0.02799897]
[ 323.84409902  298.102586   1028.23341819] [-0.19496146  0.21075588  0.0246242 ]
[-1.32199240e-01  1.49476157e+01  1.02278223e+03] [ 0.17005024 -0.1784537   0.00354099]


In [ ]:
four_points = xy[[0,721,1409,1462],:]
print(f"{four_points = }")

four_points = array([[ 315., 1996.],
       [2133.,  235.],
       [ 892.,  561.],
       [2215., 2018.]])


In [ ]:
from optv.orientation import external_calibration, full_calibration

external_calibration(cal, ref_pts, four_points, cpar)
cal.get_pos(), cal.get_angles()

(array([ 321.45823153,    3.52460111, 1044.30406926]),
 array([ 0.18811275,  0.20473034, -0.03333057]))

In [ ]:
from optv.imgcoord import image_coordinates
from optv.transforms import convert_arr_metric_to_pixel

targets = convert_arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
cpar,
)
four_points - targets

array([[ 1.07162785,  0.42689936],
       [-0.92980201, -1.04160278],
       [ 0.78724212, -0.88305194],
       [-0.96774275,  1.50665052]])

In [ ]:
def array_to_calibration(x:np.ndarray, cal:Calibration) -> None:
    cal.set_pos(x[:3])
    cal.set_angles(x[3:6])
    cal.set_primary_point(x[6:9])
    cal.set_radial_distortion(x[9:12])
    cal.set_decentering(x[12:14])
    cal.set_affine_trans(x[14:])
    return None

def calibration_to_array(cal:Calibration) -> np.ndarray:
    return np.concatenate([
        cal.get_pos(),
        cal.get_angles(),
        cal.get_primary_point(),
        cal.get_radial_distortion(),
        cal.get_decentering(),
        cal.get_affine(),
    ])

In [ ]:
def error_function(x, cal, XYZ, xy, cpar):
    
    array_to_calibration(x, cal)

    # print(np.concatenate([
    #     cal.get_pos(),
    #     cal.get_angles(),
    #     cal.get_primary_point(),
    #     cal.get_radial_distortion(),
    #     cal.get_decentering(),
    #     cal.get_affine(),
    # ]))
    
    targets = convert_arr_metric_to_pixel(
        image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
    cpar,
    )
    # err = np.sum(np.abs(xy - targets))
    err = np.sum((xy - targets)**2)
    # print(err)
    return err

In [ ]:

x0 = calibration_to_array(cal)
print(x0)


[ 3.27356601e+02  2.24449379e-01  1.05821584e+03  1.83434597e-01
  2.05537972e-01 -3.28817723e-02  1.12548584e-01 -1.69477018e-01
  3.56691868e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]


In [ ]:
sol = minimize(error_function, x0, args=(cal, XYZ, xy, cpar), method='Nelder-Mead', tol=1e-11)

In [ ]:
sol.x



array([ 3.27084305e+02,  4.23776489e-01,  1.05714134e+03,  1.83427193e-01,
        2.05148804e-01, -3.46027281e-02,  1.24764090e-01, -1.68729475e-01,
        3.56817211e+01, -3.35821643e-05, -2.31224249e-07,  8.07603352e-10,
        4.98153853e-05, -3.62128691e-05,  9.99006515e-01, -3.37053207e-03])

In [ ]:
array_to_calibration(sol.x, cal)

targets = convert_arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, cpar.get_multimedia_params()),
cpar,
)
four_points - targets


array([[ 0.43149967,  0.17439248],
       [-0.54319858,  0.79928933],
       [ 0.48987584, -0.51916561],
       [-0.56969482,  1.57124065]])

In [ ]:
targets = convert_arr_metric_to_pixel(
    image_coordinates(XYZ, cal, cpar.get_multimedia_params()),
cpar,
)


In [ ]:

import plotly.figure_factory as ff
# px.scatter(x=xy[:,0], y=xy[:,1], color=ID).show()
fig = ff.create_quiver(x=xy[:,0], y=xy[:,1], u=targets[:,0]-xy[:,0], v=targets[:,1]-xy[:,1], scale=5)
fig.show()


In [ ]:
# cal.write(working_path / "calibration" / "cam{cam_id}_scipy.ori", working_path / "calibration" / "cam{cam_id}_scipy.addpar")